In [1]:
#!pip install qiskit-terra==0.22.3 qiskit-ignis==0.6.0 qiskit-ibmq-provider==0.19.2 qiskit-aer==0.11.2 qiskit==0.39.4  cirq-core==0.14.1 cirq-google==0.14.1

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit.visualization import array_to_latex, plot_bloch_vector, plot_bloch_multivector, plot_state_qsphere, plot_state_city
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit import execute, Aer
import qiskit.quantum_info as qi
from qiskit.extensions import Initialize
#from qiskit.providers.aer import extensions  # import aer snapshot instructions
from qiskit import Aer
from qiskit_nature.drivers import UnitsType, Molecule
#from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
#from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
#from qiskit_nature.mappers.second_quantization import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
#from qiskit_nature.converters.second_quantization import QubitConverter
# Electronic Structure Problems with v0.5
# https://qiskit.org/documentation/nature/migration/00b_Electronic_structure_with_v0.5.html
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import QubitConverter
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer
#from qiskit_nature.operators.second_quantization import FermionicOp
#from qiskit_nature.circuit.library.initial_states import HartreeFock
#from qiskit_nature.circuit.library.ansatzes import UCCSD
from qiskit.providers.aer import StatevectorSimulator
from qiskit import Aer
from qiskit.utils import QuantumInstance
#from qiskit_nature.algorithms import VQEUCCFactory, GroundStateEigensolver, NumPyMinimumEigensolverFactory, BOPESSampler
#from qiskit.algorithms import NumPyMinimumEigensolver, VQE, HamiltonianPhaseEstimation, PhaseEstimation
from qiskit.algorithms import HamiltonianPhaseEstimation, PhaseEstimation
# https://github.com/Qiskit/qiskit-nature/issues/750
#from qiskit_nature.algorithms import BOPESSampler
# https://qiskit.org/documentation/nature/tutorials/03_ground_state_solvers.html
from qiskit.primitives import Estimator, Sampler
from qiskit.algorithms.optimizers import SLSQP, SPSA, QNSPSA
from qiskit_nature.second_q.algorithms import VQEUCCFactory
from qiskit_nature.second_q.circuit.library import UCCSD
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit.circuit.library import TwoLocal
from qiskit.opflow import StateFn, PauliExpectation,  CircuitSampler, PauliTrotterEvolution
from functools import partial as apply_variation
from pyscf import gto, scf
import pyqmc.api as pyq
import h5py
from ase import Atoms
from ase.build import molecule
from ase.visualize import view
import cmath
import math
import scipy.stats as stats
import qutip
import time, datetime
import pandas as pd
import os.path

# Deloitte Climate challenge 


In [3]:
def run_PySCF(info_dict, pyqmc=True, show=True):
  # Reset the files
  for fname in ['mf.hdf5','optimized_wf.hdf5','vmc_data.hdf5','dmc.hdf5']:
    if os.path.isfile(fname):
        os.remove(fname)
  
  atoms = info_dict['atoms']
  coords = info_dict['coords']
  charge = info_dict['charge']
  multiplicity = info_dict['multiplicity']
  atom_pair = info_dict['atom_pair']

  s = ''
  k = 0
  for atom in atoms:
    s += atoms[k] + ' ' + str(coords[k][0]) + ' ' + str(coords[k][1]) + ' ' + str(coords[k][2]) + '; '
    k += 1
  s = s[0:-2]
  
  mol_PySCF = gto.M(atom = s)
  
  mf = scf.RHF(mol_PySCF)
  mf.chkfile = "mf.hdf5"
  
  conv, e, mo_e, mo, mo_occ = scf.hf.kernel(mf)
  if show:
    if conv:
      print("PySCF restricted HF (RHF) converged ground-state energy: {:.12f}".format(e))
    else:
      print("PySCF restricted HF (RHF) ground-state computation failed to converge")

  if pyqmc:
    pyq.OPTIMIZE("mf.hdf5",# Construct a Slater-Jastrow wave function from the pyscf output
      "optimized_wf.hdf5", # Store optimized parameters in this file.
      nconfig=100,         # Optimize using this many Monte Carlo samples/configurations
      max_iterations=4,    # 4 optimization steps
      verbose=False)

    with h5py.File("optimized_wf.hdf5") as f:
      iter = f['iteration']
      energy = f['energy']
      error = f['energy_error']
      l = energy.shape[0]
      e = energy[l-1]
      err = error[l-1]
      if show:
        if err < 0.1:
          print("Iteration, Energy, Error")
          for k in iter:
            print("{}:         {:.4f} {:.4f}".format(k, energy[k], error[k]))
          print("PyQMC Monte Carlo converged ground-state energy: {:.12f}, error: {:.4f}".format(e, err))
        else:
          print("PyQMC Monte Carlo failed to converge")

  return conv, e

In [4]:
def U(theta):
  unitary = QuantumCircuit(1)
  unitary.p(np.pi*2*theta, 0)
  return unitary

def do_qpe(unitary, nqubits=3, show=True):
  state_in = QuantumCircuit(1)
  state_in.x(0)
  pe = PhaseEstimation(num_evaluation_qubits=nqubits, quantum_instance=quantum_instance)
  result = pe.estimate(unitary, state_in)
  phase_out = result.phase
  if show:
    print("Number of qubits: {}, QPE phase estimate: {}".format(nqubits, phase_out))
  return(phase_out)

quantum_instance = QuantumInstance(backend = Aer.get_backend('aer_simulator_statevector'))
theta = 1/2 + 1/4 + 1/8
print("theta: {}".format(theta))
unitary = U(theta)
result = do_qpe(unitary, nqubits=3)

theta: 0.875
Number of qubits: 3, QPE phase estimate: 0.875


In [5]:
quantum_instance = QuantumInstance(backend = Aer.get_backend('aer_simulator_statevector'))

numpy_solver = NumPyMinimumEigensolver()

tl_circuit = TwoLocal(rotation_blocks = ['h', 'rx'], entanglement_blocks = 'cz',
                      entanglement='full', reps=2, parameter_prefix = 'y')

# Leveraging Qiskit Runtime
# https://qiskit.org/documentation/nature/tutorials/07_leveraging_qiskit_runtime.html

#vqe_tl_solver = VQE(ansatz = tl_circuit,
                     #quantum_instance = QuantumInstance(Aer.get_backend('aer_simulator_statevector')))
estimator = Estimator()
optimizer = SPSA(maxiter=100)
vqe_tl_solver = VQE(estimator, tl_circuit, optimizer)

#vqe_ucc_solver = VQEUCCFactory(quantum_instance, ansatz = tl_circuit)
#https://qiskit.org/documentation/nature/tutorials/03_ground_state_solvers.html
vqe_ucc_solver = VQEUCCFactory(Estimator(), UCCSD(), SPSA())

qnspsa_loss = []
def qnspsa_callback(nfev, x, fx, stepsize, accepted):
    qnspsa_loss.append(fx)

# Sampling the potential energy surface
_EPS = 1e-2 # Global variable used to chop small numbers to zero
def bopes(info_dict, mapper, num_electrons, num_spatial_orbitals, two_qubit_reduction, z2symmetry_reduction,
          name_solver, perturbation_steps, qubit_converter, solver, show=True):
  
  atoms = info_dict['atoms']
  coords = info_dict['coords']
  charge = info_dict['charge']
  multiplicity = info_dict['multiplicity']
  atom_pair = info_dict['atom_pair']

  size = len(perturbation_steps)
  
  energy = np.empty(size)

  x0 = coords[atom_pair[0]][0]
  y0 = coords[atom_pair[0]][1]
  z0 = coords[atom_pair[0]][2]
  if show:
    print("x0, y0, z0 :", x0, y0, z0)
  
  x1 = coords[atom_pair[1]][0]
  y1 = coords[atom_pair[1]][1]
  z1 = coords[atom_pair[1]][2]
  if show:
    print("x1, y1, z1 :", x1, y1, z1)
  
  m = 0
  p = y0
  if abs(x1 - x0) > _EPS:
    # Find the equation of a straight line y = m*x + p that crosses the points of the atom pair
    m = (y1 - y0)/(x1 - x0)
    p = y0 - m*x0  
  
  print("Number of perturbation steps: ", size)
  for k in range(size):
    print("Step: ", k)
    
    if (abs(x0)<_EPS and abs(y0)<_EPS):
      z0_new = z0 + perturbation_steps[k]
      coords_new = []
      for l in range(len(coords)):
        if l == atom_pair[0]:
          coords_new.append((0.0, 0.0, z0_new))
        else:
          coords_new.append(coords[l])

    elif (abs(z0)<_EPS and abs(z1)<_EPS):
      x0_new = x0 + perturbation_steps[k]
      y0_new = m*x0_new + p
      coords_new = []
      for l in range(len(coords)):
        if l == atom_pair[0]:
          coords_new.append((x0_new, y0_new, 0.0))
        else:
          coords_new.append(coords[l])
    else:
        print("bopes - Error: unsupported molecule geometry, atom pairs must be in the same line or in the same plane")
        return perturbation_steps, 0
    
    info_dict_new={'atoms':atoms, 'coords':coords_new, 'charge':charge, 
                  'multiplicity':multiplicity, 'atom_pair':atom_pair}
    
    fermionic_hamiltonian, num_particles, num_spin_orbitals, qubit_op, qubit_converter, ground_state = \
    solve_ground_state(info_dict_new, mapper=mapper, num_electrons=num_electrons, num_spatial_orbitals=num_spatial_orbitals,
                  two_qubit_reduction=two_qubit_reduction, z2symmetry_reduction=z2symmetry_reduction, 
                  name_solver=name_solver, solver=solver, pyqmc=False, show=show)
    
    energy[k] = ground_state.total_energies
    
  return perturbation_steps, energy

In [6]:
def get_particle_number(problem, show=True):
  # https://qiskit.org/documentation/nature/stubs/qiskit_nature.second_q.problems.ElectronicStructureProblem.num_spin_orbitals.html
  num_spin_orbitals = problem.num_spin_orbitals
  num_particles = problem.num_particles
  
  if show:
    print("Number of particles: {}".format(num_particles))
    print("Number of spin orbitals: {}".format(num_spin_orbitals))
    
  return num_particles, num_spin_orbitals

def fermion_to_qubit(problem, second_q_op, mapper, truncate=20, two_qubit_reduction=False, z2symmetry_reduction=None, show=True): 
# Electronic Structure Problems with v0.5
# https://qiskit.org/documentation/nature/migration/00b_Electronic_structure_with_v0.5.html
# https://qiskit.org/documentation/nature/stubs/qiskit_nature.second_q.mappers.QubitConverter.html#qubitconverter
  if show:
    print("Qubit Hamiltonian operator")
  dmap = {"Jordan-Wigner": JordanWignerMapper(), "Parity": ParityMapper(), "Bravyi-Kitaev": BravyiKitaevMapper()}
  qubit_op = None
  qubit_converter = None
  for k, v in dmap.items():
    if k == mapper:
      if show:
        print("{} transformation ". format(mapper))
      qubit_converter = QubitConverter(v, two_qubit_reduction=two_qubit_reduction, z2symmetry_reduction=z2symmetry_reduction)
      # QubitConverter.convert
      # https://qiskit.org/documentation/nature/stubs/qiskit_nature.second_q.mappers.QubitConverter.convert.html
      if two_qubit_reduction:
        qubit_op = qubit_converter.convert(second_q_op, num_particles=problem.num_particles)
      else:
        qubit_op = qubit_converter.convert(second_q_op)
      n_items = len(qubit_op)
      if show:
        print("Number of items in the Pauli list:", n_items)
        if n_items <= truncate:
          print(qubit_op)
        else:
          print(qubit_op[0:truncate])
  return qubit_op, qubit_converter


# Leveraging Qiskit Runtime
# https://qiskit.org/documentation/nature/tutorials/07_leveraging_qiskit_runtime.html
def run_vqe(name, problem, qubit_converter, solver, show=True):
  calc = GroundStateEigensolver(qubit_converter, solver)
  start = time.time()
  ground_state = calc.solve(problem)
  elapsed = str(datetime.timedelta(seconds = time.time()-start))
  if show:
    print("Running the VQE using the {}".format(name))
    print("Elapsed time: {} \n".format(elapsed))
    print(ground_state)
  return ground_state

def run_qpe(qubit_op, n_ancillae=3, num_time_slices = 1, show=True):
  evolution = PauliTrotterEvolution('trotter', reps=num_time_slices)

  qpe = HamiltonianPhaseEstimation(n_ancillae, quantum_instance=quantum_instance)
  state_preparation = None
  result = qpe.estimate(qubit_op, state_preparation, evolution=evolution)

  eigv = result.most_likely_eigenvalue
  
  if show:
    print("QPE computed electronic ground state energy (Hartree): {}".format(eigv))
  
  return eigv

def plot_energy_landscape(dist, energy):
  if len(dist) > 1:
      plt.plot(dist, energy, label="VQE Energy")
      plt.xlabel('Atomic distance Deviation(Angstrom)')
      plt.ylabel('Energy (hartree)')
      plt.legend()
      display(plt.show())
  else:
      print("Total Energy is: ", energy_surface_result.energies[0], "hartree")
      print("(No need to plot, only one configuration calculated.)")
  return

def plot_loss(loss, label, target):
  plt.figure(figsize=(12, 6))
  plt.plot(loss, 'tab:green', ls='--', label=label)
  plt.axhline(target, c='tab:red', ls='--', label='target')
  plt.ylabel('loss')
  plt.xlabel('iterations')
  plt.legend()
  display(plt.show())

def solve_ground_state(
    info_dict,
    mapper ="Parity",
    num_electrons=None,
    num_spatial_orbitals=None,
    freeze_core=None, 
    two_qubit_reduction=False,
    z2symmetry_reduction = "Auto",
    name_solver='NumPy exact solver',
    solver=NumPyMinimumEigensolver(),
    plot_bopes=False,
    perturbation_steps=np.linspace(-1, 1, 3),
    pyqmc=True,
    n_ancillae=3, 
    num_time_slices=1,
    loss=[],
    label=None,
    target=None,
    show=True
):
    
    atoms = info_dict['atoms']
    coords = info_dict['coords']
    charge = info_dict['charge']
    multiplicity = info_dict['multiplicity']
    atom_pair = info_dict['atom_pair']
    
    moleculeinfo = MoleculeInfo(atoms, coords, charge=charge, multiplicity=multiplicity)
    
    # Define the electronic structure molecule driver
    # Electronic Structure Problems with v0.5
    # https://qiskit.org/documentation/nature/migration/00b_Electronic_structure_with_v0.5.html
    # https://qiskit.org/documentation/nature/tutorials/01_electronic_structure.html
    driver = PySCFDriver.from_molecule(moleculeinfo, basis="sto3g")

    # Split into classical and quantum
    if num_electrons != None and num_spatial_orbitals != None:
      # https://qiskit.org/documentation/nature/tutorials/05_problem_transformers.html
      # https://qiskit.org/documentation/nature/stubs/qiskit_nature.second_q.transformers.ActiveSpaceTransformer.html#activespacetransformer
      split = ActiveSpaceTransformer(num_electrons=num_electrons, num_spatial_orbitals=num_spatial_orbitals)
    else:
      split = None

    # Define an electronic structure problem
    problem = driver.run()
    if split != None:
      problem = split.transform(problem)
    elif freeze_core != None:
      problem = freeze_core.transform(problem)
    
    # Get the electronic energy fermionic Hamiltonian
    fermionic_hamiltonian = problem.hamiltonian
    second_q_op = fermionic_hamiltonian.second_q_op()
    
    if show:
      print("Fermionic Hamiltonian operator")
      # We print the first 20 terms of the fermionic Hamiltonian operator of the molecule
      # https://qiskit.org/documentation/nature/migration/00b_Electronic_structure_with_v0.5.html
      print("\n".join(str(second_q_op).splitlines()[:22] + ["..."]))
    
     # Get number of particles and number of spin orbitals
    num_particles, num_spin_orbitals = get_particle_number(problem, show=show)
    
    # Use the function fermion_to_qubit() to convert a fermionic operator to a qubit operator
    if show:
      print(" ")
    qubit_op, qubit_converter = fermion_to_qubit(problem, second_q_op, mapper=mapper, two_qubit_reduction=two_qubit_reduction, z2symmetry_reduction=z2symmetry_reduction, show=show)
    
    # Run the the PySCF RHF method
    if show:
      print(" ")
    conv, e = run_PySCF(info_dict, pyqmc=pyqmc, show=show)
    
    # Run QPE
    eigv = run_qpe(qubit_op, n_ancillae=n_ancillae, num_time_slices=num_time_slices, show=show)

    # Run VQE
    if show:
      print(" ")
    ground_state = run_vqe(name_solver, problem, qubit_converter, solver, show=show)
    
    # Plot loss function
    if loss != []:
      plot_loss(loss, label, target)
    
    if plot_bopes:
      # Compute the potential energy surface
      dist, energy = bopes(info_dict, mapper, num_electrons, num_spatial_orbitals, two_qubit_reduction, z2symmetry_reduction,
          name_solver, perturbation_steps, qubit_converter, solver, show=False)
      
      # Plot the energy as a function of atomic separation
      plot_energy_landscape(dist, energy)

    return fermionic_hamiltonian, num_particles, num_spin_orbitals, qubit_op, qubit_converter, ground_state

# Task 1A
- Used Chapter 5 as base
- Added atom based on exact coordinates from https://cccbdb.nist.gov/justgeom.asp
- Coordinated need to be verified
- Charge and multiplicity need to be verified
- Atom Pair is from the example
- Changed mapper to Jordan-Wigner
- Added freeze_core from example. Remove orbitals need to be thought through, what orbitals are removed?
- Added Plot_bopes to calculate PES from ground state

In [12]:
# Task 1A

mol_atoms = ["Mg", "Mg", "C", "O", "O"]
mol_coords = [(0.0, 0.0, 1.5325580),(0.0, 0.0, -1.5325580),(0.0, 0.0, 0.0),(0.0, 0.0, 1.1691750),(0.0, 0.0, -1.1691750)]

mol_charge = 1
mol_multiplicity = 2
mol_atom_pair=(2, 1)
mol_info_dict={'atoms':mol_atoms, 'coords':mol_coords, 'charge':mol_charge, 'multiplicity':mol_multiplicity, 'atom_pair':mol_atom_pair}

mol_moleculeinfo = MoleculeInfo(mol_atoms, mol_coords, charge=mol_charge, multiplicity=mol_multiplicity)

### VQE Solution with UCCSD Ansatz

In [15]:
mol_fermionic_hamiltonian, mol_num_particles, mol_num_spin_orbitals, mol_qubit_op, mol_qubit_converter, mol_ground_state = \
                  solve_ground_state(mol_info_dict, mapper ="Jordan-Wigner",
                  two_qubit_reduction=True, z2symmetry_reduction=None, 
                  name_solver = 'VQE UCCSD solver', solver = vqe_ucc_solver,
                  plot_bopes = True)

Fermionic Hamiltonian operator
Fermionic Operator
number spin orbitals=66, number terms=723938
  -88.50914441801612 * ( +_0 -_0 )
+ 0.2404527228369461 * ( +_0 -_2 )
+ 1.2807156109854003 * ( +_0 -_5 )
+ -0.5421645699408284 * ( +_0 -_11 )
+ 0.27091510592306456 * ( +_0 -_14 )
+ 0.3085313414617272 * ( +_0 -_21 )
+ 0.06745512428192812 * ( +_0 -_29 )
+ 0.25351761029150044 * ( +_0 -_30 )
+ -0.5935994541015184 * ( +_0 -_32 )
+ -88.50945406908333 * ( +_1 -_1 )
+ 0.24031309886670993 * ( +_1 -_3 )
+ -0.0008662231453853177 * ( +_1 -_4 )
+ 1.2807339224753485 * ( +_1 -_6 )
+ 0.5394310228984609 * ( +_1 -_12 )
+ -0.2605444827415635 * ( +_1 -_13 )
+ 0.17287631011912552 * ( +_1 -_19 )
+ -0.30667072150623254 * ( +_1 -_20 )
+ 0.10362552046231688 * ( +_1 -_28 )
+ -0.6069829974294585 * ( +_1 -_31 )
+ 0.2404527228369468 * ( +_2 -_0 )
...
Number of particles: (23, 22)
Number of spin orbitals: 66
 
Qubit Hamiltonian operator
Jordan-Wigner transformation 


KeyboardInterrupt: 

### Classical solution

In [14]:
mol_fermionic_hamiltonian, mol_num_particles, mol_num_spin_orbitals, mol_qubit_op, mol_qubit_converter, mol_ground_state = \
                  solve_ground_state(mol_info_dict, mapper ="Jordan-Wigner",
                  freeze_core=FreezeCoreTransformer(freeze_core=True, remove_orbitals=[4, 3]),
                  two_qubit_reduction=True, z2symmetry_reduction=None, 
                  name_solver = 'NumPy exact solver', solver = numpy_solver,
                  plot_bopes = True)

Fermionic Hamiltonian operator
Fermionic Operator
number spin orbitals=40, number terms=90168
  -10.420830479407126 * ( +_0 -_0 )
+ 0.28969246095039963 * ( +_0 -_6 )
+ 0.3279528036766686 * ( +_0 -_7 )
+ 0.44448206006059154 * ( +_0 -_15 )
+ 0.46988019052506963 * ( +_0 -_18 )
+ -10.4422540945194 * ( +_1 -_1 )
+ 0.32284884249399703 * ( +_1 -_8 )
+ -0.4529985499293985 * ( +_1 -_16 )
+ -0.12278629628891674 * ( +_1 -_17 )
+ 0.4613537608206483 * ( +_1 -_19 )
+ -8.616857626022917 * ( +_2 -_2 )
+ 0.28669974862269976 * ( +_2 -_9 )
+ 0.4176857844229477 * ( +_2 -_13 )
+ -8.576717197786857 * ( +_3 -_3 )
+ 0.5993429554920953 * ( +_3 -_10 )
+ 0.006232185018245295 * ( +_3 -_14 )
+ -8.587101697790343 * ( +_4 -_4 )
+ -0.5191594720830939 * ( +_4 -_11 )
+ -8.526461239173626 * ( +_5 -_5 )
+ 0.6701565474746457 * ( +_5 -_12 )
...
Number of particles: (10, 9)
Number of spin orbitals: 40
 
Qubit Hamiltonian operator
Jordan-Wigner transformation 
Number of items in the Pauli list: 32589
-72.98548851067082 * III

KeyboardInterrupt: 